# 要約 
このJupyter Notebookは、Kaggleのコンペティション「LLM 20 Questions」でのエージェント作成プロセスを解説しています。具体的には、言語モデルを使用して「20の質問」ゲームをプレイするための質問者（質問する役割）と回答者（回答する役割）のエージェントを構築することを目的としています。

### 問題の取り組み:
- **目的**: 「20の質問」ゲームにおいて、参加者が効率的に情報を収集し、ターゲットとなる単語をできるだけ少ない質問で推測できるようにするエージェントを設計・実装する。
- **ゲームルール**: エージェントは互いに質問と答えを通じて、最終的なキーワードを導き出します。

### 手法とライブラリ:
1. **ライブラリのインポート**: PyTorchとGemmaモデル（GoogleのオープンソースLLM）を使用し、必要なライブラリをインストールして、モデルの設定を行います。
2. **GemmaFormatterクラス**: ユーザーとモデル間の会話をフォーマットするためのクラスで、ターンの開始と終了を示すトークンを使用して構成を整えます。
3. **GemmaAgentクラス**: ゲームをプレイするための基本的なエージェントクラスで、プロンプトと応答の処理、モデルとのインタラクションの管理を行います。
4. **GemmaQuestionerAgentクラスとGemmaAnswererAgentクラス**: それぞれ質問者と回答者の動作を具現化するために、GemmaAgentを継承し、特定の機能を実装します。
5. **エージェントの管理**: `get_agent`関数を使用して、適切なエージェントを初期化し、`agent_fn`関数で観察に基づいてエージェントが適切に応答するように設定します。

最終的に、ノートブックを実行することで作成される`submission.tar.gz`ファイルには、エージェントに関連するコードが含まれ、Kaggleのコンペティションに直接提出できます。この進行状況が他の参加者にも参考になることを重視して、Gemmaの例を提供する工夫もされています。

---


# 用語概説 
以下は、Jupyter notebookの内容を基に、機械学習・深層学習の初心者がつまずきそうな専門用語の解説です。特に、実務や特定のドメインにおいてあまり知られていない用語や概念に焦点を当てています。

1. **GemmaFormatter**:
   - このクラスは、与えられたプロンプトや応答を一定のフォーマットに整形します。「ターン」とは、ユーザーやモデルがコントリビュートする一連のやり取りを指します。このフォーマッタは、会話の構造を保つためにスタートやエンドトークンを使います。

2. **Causal Language Model (CLM)**:
   - この種のモデルは、対象となる文の一部を生成する際に、その前のトークンのみを考慮する方法で学習されています。たとえば、「私が食べた果物は」は次に「りんご」や「バナナ」といった単語の生成を対象とした文脈を考慮するため、時系列データの生成に特化しています。

3. **interleave_unequal関数**:
   - 異なる長さの2つのリストを交互に取り出して新しいリストを作成する関数です。不均等なデータを処理したい場合、特に質問と回答のリストを統合する目的で使用されます。

4. **エージェント (Agent)**:
   - このコンペティション文脈におけるエージェントは、特定のタスク（この場合は「20の質問」ゲーム）を行うために設計されたプログラムであり、環境とインタラクションして学習や推論を行います。

5. **コンテキストマネージャ (Context Manager)**:
   - Pythonの`with`ステートメントと一緒に使うことで、リソースの管理（例えば、ファイルのオープンとクローズなど）を簡潔に行うための構文です。このノートブックでは、PyTorchのデフォルトのテンソルデータ型を一時的に変更するために用いられています。

6. **Few-shot learning**:
   - これは、数少ない例から学習する能力を指します。`few_shot_examples`は、Agentがどのように質問や回答を生成するかの刺激として与えられ、その分、学習の効率を上げるために使われるものです。

7. **重み (Weights)**:
   - 深層学習モデルにおける重みは、ニューラルネットワークの各接続に関連するパラメータで、モデルのパフォーマンスを調整する役割を持ちます。このノートブックでは、`GemmaForCausalLM`で重みを読み込む部分がありますが、これによりモデルがトレーニングされた内容を反映します。

8. **トークン (Token)**:
   - 自然言語処理におけるトークンとは、文や語を分析可能な最小単位に分割することを指します。通常は単語やサブワードに対応しますが、モデルが入力を扱うための形式です。

9. **Tensors**:
   - PyTorchで扱われるデータの基本的な形式で、多次元配列を表します。深層学習では、画像やテキストなど、様々なデータを処理するために用いられます。

10. **Model Configuration**:
    - モデルの設定を指し、どのようにモデルを構築し、運用するかに関する情報（たとえば、エンコーディング手法や入力のサイズなど）を含んでいます。このノートブックでは、`get_config_for_2b`や`get_config_for_7b`といった関数呼び出しで設定を取得しています。

これらの用語はいずれも、自身で実装や実務経験を積まないと直面しにくいため、理解に役立つでしょう。

---


皆さんこんにちは、私はコードをいくつかの部分に分割し、chatGPTから生成されたコメントと説明を追加しました。

このリンクにあるGemmaの例（https://www.kaggle.com/models/google/gemma/PyTorch/7b-it-quant/2）は初心者にとって役立つかもしれません。

このノートブックが役立つことを願っています。

---

このノートブックは、**LLM 20 Questions**のエージェント作成プロセスを示しています。このノートブックを実行すると`submission.tar.gz`ファイルが生成されます。このファイルは、右側の**コンペティションに提出**ヘッダーから直接送信できます。あるいは、ノートブックビュワーから*Output*タブをクリックし、`submission.tar.gz`を見つけてダウンロードします。競技のホームページの左上にある**Submit Agent**をクリックして、ファイルをアップロードし、提出を行ってください。


In [ ]:
%%bash
cd /kaggle/working
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

# Part 1: ファイル作成

- `%%writefile`は、ノートブックに次の行のすべてをファイルに書き込むよう指示します。
- `submission/main.py`は、ファイルが書き込まれるパスです。もしディレクトリ`submission`が存在しない場合は、新しく作成されます。



In [ ]:
# #%%writefile submission/main.py
# # セットアップ
import os
import sys

# Part 2: 必要なライブラリのインポートとウェイトパスの設定


In [ ]:
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from pathlib import Path

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

if os.path.exists(KAGGLE_AGENT_PATH):
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"

# Part 3: GemmaFormatterによるプロンプトのフォーマット

- `GemmaFormatter`：ユーザーとモデル間の会話をフォーマットし、ターンの開始と終了を示す事前定義されたトークンを使用します。


In [ ]:
import itertools
from typing import Iterable


class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    # 初期化
    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    # 会話の現在の状態を文字列として返します。
    def __repr__(self):
        return self._state

    # ユーザープロンプトを会話に追加するメソッド
    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self
        
    # モデルの応答を会話に追加するメソッド
    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self

    # ユーザーのターンの開始を示すメソッド
    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self

    # モデルのターンの開始を示すメソッド
    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self

    # 現在のターンの終了を示すメソッド
    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self

    # リセットメソッド
    def reset(self):
        # `_state`を空の文字列に初期化
        self._state = ""  

        # 提供された場合、システムプロンプトを追加します。
        if self._system_prompt is not None:
            self.user(self._system_prompt)  
            
        # 提供された場合、few-shotの例を適用します。
        if self._few_shot_examples is not None: 
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self

###  例


In [ ]:
# システムプロンプトとfew-shot例と共にフォーマッタを初期化
formatter = GemmaFormatter(
    system_prompt="これはシステムプロンプトです。",
    few_shot_examples=["例の質問？", "例の回答。"]
)

# ユーザーターンを追加
formatter.user("フランスの首都はどこですか？")

# モデルターンを追加
formatter.model("フランスの首都はパリです。")

# フォーマットされた会話を出力
print(formatter)

`GemmaFormatter`クラスは、会話を一貫した方法で構造化およびフォーマットするのに役立ち、ターンが適切にマークされ、整理されることを保証します。

# Part 4: エージェント定義とユーティリティ

- `_set_default_tensor_type`コンテキストマネージャは、一時的にPyTorchのテンソルのデフォルトデータ型を指定されたタイプに設定し、そのブロックのコードが実行された後に再びtorch.floatにリセットされます。


In [ ]:
import re

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """指定されたdtypeにtorchのデフォルトdtypeを設定します。"""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

### 例


In [ ]:
torch.tensor([1.0, 2.0]).dtype

In [ ]:
with _set_default_tensor_type(torch.float64):
    print(torch.tensor([1.0, 2.0]).dtype)

# Part 5: 基本GemmaAgentクラス

GemmaAgentクラスは以下を目的としています:

- 言語モデルを初期化・設定します。
- プロンプトと応答をフォーマットおよび処理します。
- テンソルデータタイプを一時的に設定するためにコンテキストマネージャを使用します。
- フォーマットされたプロンプトに基づいてモデルと対話し、応答を生成します。


In [ ]:
class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        print("モデルを初期化しています")
        
        # モデル設定
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant
        
        # モデル
        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt')
            model.load_weights(ckpt_path)
            self.model = model.to(self._device).eval()

    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(f"{response=}")
        return response

    def _start_session(self, obs: dict):
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,
                'top_p': 0.1,
                'top_k': 1,
        }
        response = self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )
        return response

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower()
        return keyword

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError

# Part 6: GemmaQuestionerAgentクラス


In [ ]:
def interleave_unequal(x, y):
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]

class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("20の質問をプレイしましょう。あなたは質問者の役割を果たします。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("はいかいいえで答えてください質問をしてください。")
        elif obs.turnType == 'guess':
            self.formatter.user("今、キーワードを推測してください。推測は二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None:
                question = "それは人ですか？"
            else:
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else:
            raise ValueError("未知のターンタイプ: ", obs.turnType)

GemmaQuestionerAgent:
- `__init__`：親クラスのコンストラクタを呼び出してエージェントを設定します。
- `_start_session`：質問と回答を交互に並べ、会話のフォーマットを設定します。
- `_parse_response`：エージェントが質問をしているのか、推測をしているのかによってモデルの応答を異なる方法で解釈します。

# Part 7: GemmaAnswererAgentクラス


In [ ]:
class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"20の質問をプレイしましょう。あなたは回答者の役割を果たします。キーワードは{obs.keyword}で、カテゴリは{obs.category}です。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"このキーワードは{obs.keyword}に関連する質問です。はいかいいえで答えてください。回答は二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'

# Part 8: エージェントの作成と関数定義


In [ ]:
# エージェントの作成
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを考え、質問者からのはいかいいえの質問に応じます。キーワードは特定の人物、場所、または物です。"

few_shot_examples = [
    "20の質問をプレイしましょう。あなたは質問者の役割を果たします。最初の質問をしてください。",
    "それは人ですか？", "**いいえ**",
    "それは場所ですか？", "**はい**",
    "それは国ですか？", "**はい** ではキーワードを推測してください。",
    "**フランス**", "正解！",
]

# **重要:** エージェントをグローバルに定義してくださいので、必要なエージェントだけをロードできます。両方をロードするとおそらくOOM（メモリ不足）になります。

# エージェント変数を初期化
agent = None

# 名前に基づいて適切なエージェントを取得する関数
def get_agent(name: str):
    global agent
    
    # エージェントが初期化されておらず、要求されたエージェントが"質問者"の場合
    if agent is None and name == 'questioner':
        # 特定のパラメータでGemmaQuestionerAgentを初期化
        agent = GemmaQuestionerAgent(
            device='cuda:0',  # 計算用のデバイス
            system_prompt=system_prompt,  # エージェントのシステムプロンプト
            few_shot_examples=few_shot_examples,  # エージェントの行動を指導するための例
        )
    # エージェントが初期化されておらず、要求されたエージェントが"回答者"の場合
    elif agent is None and name == 'answerer':
        # 同じパラメータでGemmaAnswererAgentを初期化
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    
    # エージェントが初期化されていることを確認
    assert agent is not None, "エージェントが初期化されていません。"

    # 初期化されたエージェントを返す
    return agent

# 観察に基づいてインタラクションを処理するための関数
def agent_fn(obs, cfg):
    # 観察が質問するためのものである場合
    if obs.turnType == "ask":
        # 質問するエージェントを取得して観察に応答する
        response = get_agent('questioner')(obs)
    # 観察が推測するためのものである場合
    elif obs.turnType == "guess":
        # 質問するエージェントを取得して観察に応答する
        response = get_agent('questioner')(obs)
    # 観察が回答を提供するためのものである場合
    elif obs.turnType == "answer":
        # 回答するエージェントを取得して観察に応答する
        response = get_agent('answerer')(obs)
    
    # エージェントからの応答がNoneまたは非常に短いものである場合
    if response is None or len(response) <= 1:
        # ポジティブな応答（"はい"）を仮定
        return "はい"
    else:
        # エージェントから受け取った応答を返す
        return response

1. **GemmaFormatterクラス**: このクラスは、ゲームのプロンプトのフォーマットを扱います。ユーザーとモデルのターンを構成し、ターンの開始・終了、状態のリセット、ターンの適用に関するメソッドを持っています。エージェントのためのプロンプトの一貫したフォーマットを保証します。

2. **GemmaAgentクラス**: ゲーム内の一般的なエージェントを表す抽象クラスです。初期化、呼び出し、セッションの開始、言語モデル（LLM）の呼び出し、応答の解析、デフォルトのテンソルタイプの設定など、共通のメソッドと属性を定義します。

3. **GemmaQuestionerAgentクラス**と**GemmaAnswererAgentクラス**: これらのクラスはGemmaAgentから継承し、それぞれ質問者と回答者エージェントの特定の動作を実装します。ターンタイプに応じてモデルのレスポンスをカスタマイズするために、`_start_session`と`_parse_response`メソッドをオーバーライドします。

4. **interleave_unequal関数**: この関数は、長さの異なる2つのリストを交互に並べます。ゲーム内で質問と回答を交互にするために使用されます。

5. **get_agent関数**: この関数は、入力名（「questioner」または「answerer」）に基づいて適切なエージェントを初期化し、返します。エージェントは一度だけ作成され再利用されることが保証されます。

6. **agent_fn関数**: この関数は、ゲームのエントリーポイントとして機能します。観察のターンタイプに応じて使用するエージェントのタイプを決定し、適切なエージェントの`__call__`メソッドを呼び出して応答を生成します。


In [ ]:
# !apt install pigz pv > /dev/null

In [ ]:
# !tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2

---

# コメント 

> ## Mohamed MZAOUALI
> 
> より良く理解できました、どうもありがとう!
> 
> 

---